In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 
from itertools import islice
%matplotlib inline

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn

## Getting the MNIST Data Set

We have access to a number of now standard datasets via PyTorch directly.  So to get MNIST set up, all we need to do is the following.

In [ ]:
image_path = './' # just store everything to your local working directory
mnist_data_path = image_path + '/MNIST'

transform = transforms.Compose([transforms.ToTensor()])

if not os.path.isdir(mnist_data_path): # if you've already downloaded and converted the MNIST images, skip

    # Download and convert MNIST images into tensor format. 
    mnist_dataset = torchvision.datasets.MNIST(image_path, train=True, transform=transform, download=True)
    mnist_dataset_test = torchvision.datasets.MNIST(image_path, train=False, transform=transform, download=True)
     # Sanity check to make sure your download worke
    assert isinstance(mnist_dataset, torch.utils.data.Dataset)

else:
    # Download and convert MNIST images into tensor format. 
    mnist_dataset = torchvision.datasets.MNIST(image_path, train=True, transform=transform, download=False)
    mnist_dataset_test = torchvision.datasets.MNIST(image_path, train=False, transform=transform, download=False)

mnist_dataset_valid = Subset(mnist_dataset, torch.arange(10000))
mnist_dataset_train = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))


In [ ]:
fig = plt.figure(figsize=(15,6))
for ii, (image, label) in islice(enumerate(mnist_dataset_train),10):
    ax = fig.add_subplot(2,5,ii+1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(torch.squeeze(image,0), cmap='gray_r')
    ax.set_title(f'{label}', size=15)

Here we decide on a batch size for running Stochastic Gradient Descent and then get our training data into the proper format for iterating over epochs with a random shuffle into new batches at every iteration.  

In [ ]:
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_dataset_train, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_dataset_valid, batch_size, shuffle=False)

## Building the Model

In [ ]:
model = nn.Sequential()

# First convoultional/pooling layer
model.add_module(
    'conv1',
    nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
    )
)
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))

# Second convoultional/pooling layer
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

# Fully connected classifier label like we had before
model.add_module('flatten', nn.Flatten())

# We need to know how large of a vector our model will produce,
# so we introduce a little hack.

x = torch.ones((4, 1, 28, 28))
dims = model(x).shape

# Now that we know the right sizes to use, we build our 
# connected network with droput before we get to the 
# classifier.

model.add_module('fc1', nn.Linear(dims[1], 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10))

## Choosing the Loss Function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## Training across Epochs

In [ ]:
num_epochs = 20
loss_history_train = np.zeros(num_epochs)
accuracy_history_train = np.zeros(num_epochs)
loss_history_valid = np.zeros(num_epochs)
accuracy_history_valid = np.zeros(num_epochs)

for epoch in range(num_epochs):
    model.train() 
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward() # perform backpropagation
        optimizer.step() # optimize over the given batch
        optimizer.zero_grad() # zero out the gradient for the next iteration
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        loss_history_train[epoch] += loss.item()*y_batch.size(0)
        accuracy_history_train[epoch] += is_correct.sum()

    loss_history_train[epoch] /= len(train_dl.dataset)
    accuracy_history_train[epoch] /= len(train_dl.dataset)
    model.eval() # Note, we need model.train() and model.eval() to manage the dropout layer correctly 
    
    # We now look at validation data as we train to get a sense of how generalizable our model is to test data.

    with torch.no_grad(): # don't optimize at this point, i.e. keep all weights fixed 
        for x_batch, y_batch in valid_dl:
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            loss_history_valid[epoch] += loss.item()*y_batch.size(0)
            accuracy_history_valid[epoch] += is_correct.sum()
    loss_history_valid[epoch] /= len(valid_dl.dataset)
    accuracy_history_valid[epoch] /= len(valid_dl.dataset)
    
    print(f"Epoch {epoch} accuracy: {accuracy_history_train[epoch]:.4f} "
          f"val_accuracy: {accuracy_history_valid[epoch]}")


In [ ]:
x_arr = np.arange(num_epochs)
fig = plt.figure(figsize=(12,4))
ax=fig.add_subplot(1,2,1)
ax.plot(x_arr, loss_history_train, '-o', label='Training Loss')
ax.plot(x_arr, loss_history_valid, '-o', label='Validation Loss')
ax.legend()

ax=fig.add_subplot(1,2,2)
ax.plot(x_arr, accuracy_history_train, '-o', label='Training Accuracy')
ax.plot(x_arr, accuracy_history_valid, '-o', label='Validation Accuracy')
ax.legend()

## Testing Our Trained Model 

In [ ]:
test_data = mnist_dataset_test.data/255.
test_pred = model(test_data.unsqueeze(1))
is_correct = (torch.argmax(test_pred, dim=1) == mnist_dataset_test.targets ).float()
print(f'Test Accuracy is: {is_correct.mean():.4f}')